# Read Image data into memory

In [1]:
import time
import getpass
import warnings
warnings.filterwarnings('ignore')

username = getpass.getuser()
start_time = time.time()
#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/charu/intuit/code/face/data/lfw"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    result = filename.split("/") 
    labels.append(result[len(result)-2]) #Get the label name from filename
    
doForEachFile(collectFileNamesAndLabels,baseDir, 500)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 1.9316399097442627 seconds
Files collected :  325


In [2]:
start_time = time.time()
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))
print("Total data collection time = %s seconds" % (time.time() - start_time))

Images data collected :  325
Total data collection time = 0.7101178169250488 seconds


In [3]:
#%run "TransferLearningWithCNN.ipynb"

# Generic Multi threading code

Used later

In [4]:
import threading
import sys
import traceback
import time

class myThread (threading.Thread):
    def __init__(self, threadName, index, worker):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.index = index
        self.worker = worker
    def run(self):
        print('^',self.threadName, ', ', end='')
        try:
            self.worker(self.threadName, self.index)
        except:
            e = sys.exc_info()[0]
            print("Exception in thread: ",self.threadName,", ", e)
            traceback.print_exc()
        print('V',self.threadName, end='')
        

def runThreads(nThreads, worker):
    threads = []
    for i in range(nThreads):
        try:
            t = myThread("Thread-"+str(i), i , worker)
            threads.append(t)
            t.start()
        except:
            e = sys.exc_info()[0]
            print("Error: unable to start thread: ", e)
            traceback.print_exc()

    time.sleep(1)

    for t in threads:
        t.join()

In [5]:
start_time = time.time()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


Y = pd.DataFrame(labels, columns = ['name'])
sizeOfYLabel = len(Y)
oneDImageList = []
showOneImage = False

imageListLock = threading.Lock()

def flattenImage(image, imageList):
    # Convert RGB to single number
    global showOneImage
    if showOneImage:
        plt.imshow(image, cmap="Greys")
        plt.show()
    gray = rgb2gray(image)
    if showOneImage:
        plt.imshow(gray, cmap="Greys")
        plt.show()
        showOneImage = False
    imageList.append(gray.flatten())

nThreads = 10
splitImagesList = np.array_split(imagesList, nThreads)
splitLabelsList = np.array_split(labels, nThreads)
labels = []

def imageFlatteningWorker(threadName, index):
    global splitImagesList
    global imageListLock
    imageList = []
    for image in splitImagesList[index]:
        flattenImage(image, imageList)
    imageListLock.acquire()
    global oneDImageList
    oneDImageList.extend(imageList)
    global labels
    labels.extend(splitLabelsList[index])
    imageListLock.release()

runThreads(nThreads, imageFlatteningWorker)
    
del imagesList

X = pd.DataFrame(oneDImageList)
del oneDImageList
print("X shape", X.shape)
print("label count", len(labels))
print("Total dataframe creation time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^ Thread-3 , ^ Thread-4 , ^ Thread-5 , ^ Thread-6 , ^^ Thread-8 ,  ^ Thread-9 , Thread-7 , V Thread-9V Thread-3V Thread-7V Thread-1V Thread-8V Thread-5V Thread-0V Thread-4V Thread-2V Thread-6X shape (325, 62500)
label count 325
Total dataframe creation time = 6.972318887710571 seconds


In [6]:
#Y['name'].value_counts()

In [7]:
start_time = time.time()
maxFrequency = 21

# We upsample/downsample to have equal quantity of all classes

#X_sample = pd.DataFrame(columns = X.columns)
#Y_sample = pd.DataFrame(columns = Y.columns)
X_sampleList = []
Y_sampleList = []

XY_sampleLock = threading.Lock()

def equalSampleLabel(label):
    global X_sampleList
    global Y_sampleList
    global XY_sampleLock
    dfa = X[Y['name']== label]
    dfasample = dfa.sample(n=maxFrequency,replace=True)
    dfb = Y[Y['name']== label]
    dfbsample = dfb.sample(n=maxFrequency,replace=True)
    print('.', end='')
    XY_sampleLock.acquire()
    print('-', end='') 
    X_sampleList.append(dfasample)
    Y_sampleList.append(dfbsample)
    XY_sampleLock.release()
    print('+', end='')



labels = Y['name'].unique()

nThreads = 10
labelsArray = np.array_split(labels, nThreads)

def worker(threadName, index):
    for label in labelsArray[index]:
        #print(threadName, label)
        equalSampleLabel(label)

runThreads(nThreads, worker)

X_sample = pd.concat(X_sampleList)
Y_sample = pd.concat(Y_sampleList)
print(X_sample.shape)
print(Y_sample.shape)
X = X_sample
Y = Y_sample
# print(X_sample)
print("Total equal sampling time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 ^ Thread-2 , , ^ Thread-3 , ^ Thread-4 ^ Thread-5 , , ^ Thread-6 , .-^ Thread-7 , +^ Thread-8 , ^ Thread-9 , .-.+..-.+-+...-+-+-+-+.-..+..-+-+-+-..+-..+-+-...+-+..-..+-+-+-+-+.-+-...+..-.+-+-+-+-.+.-+-+-+-+-.+.-+.-+.-+.-+-+.-.+.-+.-+.-+-+...-+-.+-....+-+.-+.-.+-.+-+-+..-+.-+.-+-+-+-+-+.-+..-+-+.-+.-+.-+.-+....-+.-...+.-.+-+-+.-+-+-+.-+-+.-+-.+-.+-+-+..-+-..+.-...+-+.-..+-+.-.+.-+-.+-+.-+-+.-+-+-.+-+-....+-+-+-+-+-+.-.+.-+.-+-+.-+..-+.-.+.-...+.-+-+-+-+-.+..-+-+-+-+..-+-+..-+..-+-+-...+-+-+.-....+-.+-+-.+.-+-.+-+-.+.-.+-+-.+-.+-+-+-..+-..+-..+-.+-+-+-.+...-+-+-+-+.-+-.+-+.-+.-.+-+......-+-+V-+ Thread-3V Thread-2-+V Thread-4..-+-.+.-+V Thread-8-+-+.-+V Thread-7-+.-+.-+V Thread-1..-+V Thread-0-+V Thread-5.-+V Thread-6.-+V Thread-9(3717, 62500)
(3717, 1)
Total equal sampling time = 2.8505489826202393 seconds


In [8]:
start_time = time.time()
sizeOfX = X_sample.shape[1]
#%run "DNN.ipynb"

print("Total DNN time = %s seconds" % (time.time() - start_time))

Total DNN time = 6.699562072753906e-05 seconds


In [9]:
print(X_sample.shape)


(3717, 62500)


In [10]:
# print(Y)
# enc.get_feature_names(['name'])
# print(enc.get_feature_names(['name']).shape)

In [11]:
# import numpy as np
# import tensorflow as tf

# Ypred = pd.DataFrame( columns = ['sampleId', "classAsIndex", "classAsString", "actualClass"])
# for i in range(300):
#     prediction = model.predict(X_sample.iloc[i:i+1,:],verbose=0)
#     class_labels = np.argmax(prediction[0], axis=0)
#     Ypred.at[i, 'sampleId']  = i
#     Ypred.at[i, 'classAsIndex']  = class_labels
#     Ypred.at[i, 'classAsString']  = enc.get_feature_names(['name'])[class_labels]
#     Ypred.at[i, 'actualClass']  = Y_sample.iloc[i].values
#     #print(class_labels, enc.get_feature_names(['name'])[class_labels]," Names of class :: ", Y_sample.iloc[i].values)

# from IPython.display import display, HTML
# display(HTML(Ypred.to_html()))

In [12]:

class_mapping = { i : labels[i] for i in range(0, len(labels) ) }
print(class_mapping)

inv_class_mapping = {v: k for k, v in class_mapping.items()}
print(inv_class_mapping)
# print(type(inv_class_mapping))
# print(inv_class_mapping['0'])

{0: 'German_Khan', 1: 'Stefano_Gabbana', 2: 'Dragan_Covic', 3: 'Jeff_Hornacek', 4: 'Sureyya_Ayhan', 5: 'Deb_Santos', 6: 'Bob_Newhart', 7: 'Wang_Hailan', 8: 'Paul_McNulty', 9: 'Jimmy_Iovine', 10: 'Claudia_Pechstein', 11: 'Ranil_Wickremasinghe', 12: 'Ben_Chandler', 13: 'Mark_Komara', 14: 'Rand_Beers', 15: 'Joanne_Woodward', 16: 'John_Bond', 17: 'Reginald_Hudlin', 18: 'Lee_Baca', 19: 'Mary-Kate_Olsen', 20: 'Emily_Stevens', 21: 'Xiang_Huaicheng', 22: 'Phil_Mickelson', 23: 'Gerry_Kelly', 24: 'Salma_Hayek', 25: 'Jim_Edmonds', 26: 'Martina_McBride', 27: 'Anthony_Pico', 28: 'Jose_Theodore', 29: 'Heidi_Fleiss', 30: 'Mark_Richt', 31: 'Mike_Smith', 32: 'Paul_ONeill', 33: 'Horace_Newcomb', 34: 'Mohammad_Fares', 35: 'JK_Rowling', 36: 'Shinya_Taniguchi', 37: 'Tony_Curtis', 38: 'Laurie_Pirtle', 39: 'Angel_Maza', 40: 'Ray_Sherman', 41: 'Nancy_Powell', 42: 'Kathryn_Tucker', 43: 'Gro_Harlem_Brundtland', 44: 'Jacqueline_Marris', 45: 'Mikhail_Gorbachev', 46: 'Takuma_Sato', 47: 'Ray_Lewis', 48: 'Hank_Aaron

In [13]:
from PIL import Image
import pandas as pd
import tensorflow as tf
print(X_sample.shape)
sample_dict = {}

X_transfer_learning = pd.DataFrame()

for ind in X_sample.index: 
    X_sample_array = X_sample.iloc[ind].as_matrix()
    arr = X_sample_array.reshape(250,250,1)
    img = tf.keras.preprocessing.image.array_to_img(arr)
    img.resize(size=(299,299), resample=Image.BICUBIC)
    array = tf.keras.preprocessing.image.img_to_array(img)
    sample_dict[0] = array
    X_transfer_learning = X_transfer_learning.append(sample_dict,ignore_index=True)

print(X_transfer_learning.shape)
print(X_transfer_learning[0][0].shape)
print(type(X_transfer_learning))
# Swap rows and columns of dataframe
# X_transfer_learning = X_transfer_learning.transpose()
# print(X_transfer_learning.shape)

(3717, 62500)
(3717, 1)
(250, 250, 1)
<class 'pandas.core.frame.DataFrame'>


In [14]:
label_y_sample = Y_sample.copy()
label_y_sample['name'] = label_y_sample['name'].map(inv_class_mapping)
class_images = label_y_sample['name'].value_counts()

In [15]:
print(Y_sample.shape)
print(Y_sample)
start_time = time.time()
sizeOfX = X_sample.shape[1]
print("Shape of Input to Transfer learning: X: ", X_transfer_learning.shape, ", Y: ", Y_sample.shape)
%run "TransferLearning-New.ipynb"

print("Total transfer learning with CNN time = %s seconds" % (time.time() - start_time))

(3717, 1)
                   name
0           German_Khan
0           German_Khan
0           German_Khan
0           German_Khan
0           German_Khan
..                  ...
324  Queen_Elizabeth_II
324  Queen_Elizabeth_II
323  Queen_Elizabeth_II
324  Queen_Elizabeth_II
323  Queen_Elizabeth_II

[3717 rows x 1 columns]
Shape of Input to Transfer learning: X:  (3717, 1) , Y:  (3717, 1)


Using TensorFlow backend.


valid data shapes, X_valid:  (3717, 299, 299, 3) , y_valid:  (3717,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
___________________________________________________________________________________________

Total params: 55,873,736
Trainable params: 55,813,192
Non-trainable params: 60,544
__________________________________________________________________________________________________


{'Conv2d_1a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0' shape=(?, 149, 149, 32) dtype=float32>, 'Conv2d_2a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0' shape=(?, 147, 147, 32) dtype=float32>, 'Conv2d_2b_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0' shape=(?, 147, 147, 64) dtype=float32>, 'MaxPool_3a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0' shape=(?, 73, 73, 64) dtype=float32>, 'Conv2d_3b_1x1': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0' shape=(?, 73, 73, 80) dtype=float32>, 'Conv2d_4a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_4a_3x3/Relu:0' shape=(?, 71, 71, 192) dtype=float32>, 'MaxPool_5a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_5a_3x3/MaxPool:0' shape=(?, 35, 35, 192) dtype=float32>, 'Mixed_5b': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5b/concat:0' shape=(?, 35, 35, 256) dtype=float32>, 'Mixed_5c': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5c/concat:0' shape=(?, 35, 35,

ResourceExhaustedError: OOM when allocating tensor with shape[3717,147,147,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node Mkl2Tf/_1146}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Total transfer learning with CNN time = 2450.0074620246887 seconds


In [16]:
print(test_acc)
print(X_test, y_test)
print(X_test[0])
print(type(X_test[0]))
print(type(X_train[0]))
print(n_iterations)

NameError: name 'test_acc' is not defined